In [1]:
%load_ext sql

Found pyproject.toml from '/Users/anaelle.laurans/perso/data-engineering-sandbox'

The 'toml' package isn't installed. To load settings from the pyproject.toml file, install with: pip install toml

In [3]:
%config SqlMagic.dsn_filename = "connection.ini"
%sql --section pg
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

## List schema and tables with comments on the database server

In [4]:
%%sql
SELECT table_catalog,
    table_schema,
    table_name,
    table_type,
    obj_description(pgc.oid, 'pg_class'),
    pgc.relowner,
    u.usename
FROM information_schema.tables t
    INNER JOIN pg_catalog.pg_class pgc ON t.table_name = pgc.relname
    INNER JOIN pg_catalog.pg_user u ON (pgc.relowner = u.usesysid)
WHERE table_schema NOT IN ('pg_catalog', 'information_schema')
ORDER BY table_catalog,
    table_schema,
    table_name

Running query in 'pg'

18 rows affected.

table_catalog,table_schema,table_name,table_type,obj_description,relowner,usename
dev,airbnb,adresses,BASE TABLE,None,10,myuser
dev,airbnb,hosts,BASE TABLE,None,10,myuser
dev,airbnb,listings,BASE TABLE,None,10,myuser
dev,airbnb,reviews,BASE TABLE,None,10,myuser
dev,analytics,accounts,BASE TABLE,None,10,myuser
dev,analytics,customer,BASE TABLE,None,10,myuser
dev,analytics,customersXaccount,BASE TABLE,None,10,myuser
dev,analytics,transactions,BASE TABLE,None,10,myuser
dev,analytics,transactionsMetadata,BASE TABLE,None,10,myuser
dev,geospatial,shipwrecks,BASE TABLE,None,10,myuser


## Retrieving the schema for tables

In [8]:
%%sql
SELECT '"' || table_catalog || '"."' || table_schema || '"."' || table_name || '"' AS full_name,
    column_name,
    data_type,
    column_default,
    is_nullable,
    character_maximum_length,
    numeric_precision,
    datetime_precision,
    pgd.description
FROM information_schema.columns C
    LEFT JOIN pg_catalog.pg_class pgc ON C .table_name = pgc.relname
    LEFT JOIN pg_catalog.pg_description pgd ON C .ordinal_position = pgd.objsubid
    AND pgc.oid = pgd.objoid
WHERE table_schema NOT IN ('pg_catalog', 'information_schema') AND table_schema = 'mflix';

Running query in 'pg'

39 rows affected.

full_name,column_name,data_type,column_default,is_nullable,character_maximum_length,numeric_precision,datetime_precision,description
"""dev"".""mflix"".""users""",preferences,jsonb,None,YES,None,None,None,None
"""dev"".""mflix"".""movies""",runtime,integer,None,YES,None,32,None,None
"""dev"".""mflix"".""theaters""",theaterId,integer,None,NO,None,32,None,None
"""dev"".""mflix"".""theaters""",location,jsonb,None,NO,None,None,None,None
"""dev"".""mflix"".""movies""",awards,jsonb,None,YES,None,None,None,None
"""dev"".""mflix"".""movies""",imdb,jsonb,None,YES,None,None,None,None
"""dev"".""mflix"".""movies""",released,timestamp without time zone,None,YES,None,None,6,None
"""dev"".""mflix"".""movies""",metacritic,integer,None,YES,None,32,None,None
"""dev"".""mflix"".""movies""",lastupdated,timestamp without time zone,None,YES,None,None,6,None
"""dev"".""mflix"".""movies""",year,integer,None,YES,None,32,None,None


### Table sizes

In [10]:
%%sql
SELECT table_schema,
    table_name,
    pg_size_pretty(
        pg_total_relation_size(
            '"' || table_schema || '"."' || table_name || '"'
        )
    ) AS table_size,
    pg_size_pretty(
        pg_indexes_size(
            '"' || table_schema || '"."' || table_name || '"'
        )
    ) AS index_size,
    n_live_tup AS estimated_row_count
FROM information_schema.tables t
    INNER JOIN pg_catalog.pg_stat_user_tables psut ON psut.schemaname = t.table_schema
    AND psut.relname = t.table_name
WHERE table_schema NOT IN ('pg_catalog', 'information_schema');

Running query in 'pg'

18 rows affected.

table_schema,table_name,table_size,index_size,estimated_row_count
airbnb,listings,168 kB,16 kB,26
airbnb,reviews,368 kB,48 kB,761
airbnb,adresses,32 kB,16 kB,26
airbnb,hosts,64 kB,16 kB,26
analytics,transactions,8576 kB,1944 kB,88152
analytics,transactionsMetadata,280 kB,104 kB,1746
analytics,accounts,400 kB,104 kB,1746
analytics,customersXaccount,208 kB,56 kB,1746
analytics,customer,240 kB,40 kB,500
supplies,sales,1224 kB,264 kB,5019


## Table profiling

In [19]:
%sqlcmd profile --table geospatial.shipwrecks

,id,recrd,vesslterms,feature_type,chart,latdec,londec,gp_quality,depth,sounding_type,history,quasou,watlev,coordinates
count,11095,11095,11095,11095,11095,11095,11095,11095,11095,11095,11095,11095,11095,11095
unique,11095,1,121,5,2201,11074,11084,1,243,6,139,7,6,11094
top,578f6fa2df35c7fbdbaed8c4,,,"Wrecks - Submerged, dangerous","US,US,graph,Chart 11372",nan,nan,,,,,depth unknown,always under water/submerged,"[-76.8978558, 34.498995]"
freq,1,11095,10975,6578,87,nan,nan,11095,8518,10949,10724,5573,8399,2
mean,nan,nan,nan,nan,nan,35.3897,-83.8709,nan,nan,nan,nan,nan,nan,nan
std,nan,nan,nan,nan,nan,8.4173,32.0931,nan,nan,nan,nan,nan,nan,nan
min,nan,nan,nan,nan,nan,-64.7809,-179.149,nan,nan,nan,nan,nan,nan,nan
25%,nan,nan,nan,nan,nan,29.4601,-90.2811,nan,nan,nan,nan,nan,nan,nan
50%,nan,nan,nan,nan,nan,36.9617,-81.0203,nan,nan,nan,nan,nan,nan,nan
75%,nan,nan,nan,nan,nan,40.9457,-74.5457,nan,nan,nan,nan,nan,nan,nan
